In [ ]:
! pip install nni # install nni
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip # download ngrok and unzip it
! unzip ngrok-stable-linux-amd64.zip
! mkdir -p nni_repo
! git clone https://github.com/microsoft/nni.git nni_repo/nni # clone NNI's offical repo to get examples

In [ ]:
!pip install boto3 pyyaml h5py nni[SMAC]

In [ ]:
%tensorflow_version 2.x

In [ ]:
import os
import boto3
import yaml
from google.colab import files

In [ ]:
image_path = "output/"

In [ ]:
with open("config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)        
    except yaml.YAMLError as e:
        print(e)


s3 = boto3.resource("s3", 
                    aws_access_key_id = config["S3"]["AWS_ACCESS_KEY_ID"],
                    aws_secret_access_key = config["S3"]["AWS_SECRET_ACCESS_KEY"]
                    )
bucket = s3.Bucket(config["S3"]["BUCKET"])

for bucket_object in bucket.objects.all():
    try:
        filename = bucket_object.key.strip()
        if filename.startswith(image_path) and not(filename.endswith(".json")):
            if len(filename.split('/')) == 4:
                _, folder, gemstone, name = filename.split('/')
                # Check if is a file
                if name != '':
                    path = "/".join(filename.split("/")[0:3])
                    os.makedirs(path, exist_ok=True)
                    bucket.download_file(filename, filename)
    except Exception as e:
        print("Can't download file: " + filename)
        print(e)


In [ ]:
! ./ngrok authtoken "ngrok token"

### Start experiment

In [ ]:
! nnictl create --config config.yml --port 5000 &
get_ipython().system_raw('./ngrok http 5000 &')

In [ ]:
! curl -H "Content-Type: application/json" -s http://localhost:4040/api/tunnels # don't change the port number 4040

In [ ]:
while True:pass

### Compress files

In [ ]:
%%bash

for dir in /content/2022*/   # list directories in the form "/tmp/dirname/"
do       
    dir=${dir%*/} 
    zip -r "${dir}.zip" "${dir}"
done

for dir in /content/*.h5.zip   # list directories in the form "/tmp/dirname/"
do       
    zip -r "models.zip" "${dir}"
done


In [ ]:
! zip -r /content/nni_experiments.zip /root/nni-experiments/

In [ ]:
files.download( "//content/nni_experiments.zip")

### Stop experiment

In [ ]:
# ! nnictl stop --all